In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import re 
import os
import time
import xlsxwriter
import csv
import calendar
import itertools
from itertools import chain
from os import walk
from os import listdir
from os.path import isfile, join
from functools import reduce

In [2]:
'''functions used by the following script'''
#expand user month input, get filenames to read for the next step
def get_month_input():
    expand_range = [] #expand dash
    for i in month_input.split():
        if '-' not in i:
            expand_range.append([int(i)])
        if '-' in i:
            a, b = i.split('-')
            expand_range.append(list(range(int(a), int(b)+1)))
    month_selection = list(chain.from_iterable(expand_range))
    
    #get filenames
    lottr_inter_csv_names = []
    lottr_noninter_csv_names = []
    tttr_inter_csv_names = []
    for i in month_selection:
        lottr_inter_csv_names.append('lottr_inter_'+str(year)+'_'+month_num_abbr_lookup[i])
        lottr_noninter_csv_names.append('lottr_noninter_'+str(year)+'_'+month_num_abbr_lookup[i])
        tttr_inter_csv_names.append('tttr_inter_'+str(year)+'_'+month_num_abbr_lookup[i])
    
    return (lottr_inter_csv_names, lottr_noninter_csv_names, tttr_inter_csv_names, month_selection)

#function for merging columns belong to the the same time segment for both lottr and tttr
def merge_column(res_df, merge_df, lottr_or_tttr): 
    if lottr_or_tttr == 'lottr': 
        res_df[LOTTR_AMP] = list(merge_df[lottr_amp_union_col_name].itertuples(index=False, name=None))
        res_df[LOTTR_MIDD] = list(merge_df[lottr_midd_union_col_name].itertuples(index=False, name=None))
        res_df[LOTTR_PMP] = list(merge_df[lottr_pmp_union_col_name].itertuples(index=False, name=None))
        res_df[LOTTR_WE] = list(merge_df[lottr_we_union_col_name].itertuples(index=False, name=None))
        return res_df
    elif lottr_or_tttr == 'tttr':
        res_df[TTTR_AMP] = list(merge_df[tttr_amp_union_col_name].itertuples(index=False, name=None))
        res_df[TTTR_MIDD] = list(merge_df[tttr_midd_union_col_name].itertuples(index=False, name=None))
        res_df[TTTR_PMP] = list(merge_df[tttr_pmp_union_col_name].itertuples(index=False, name=None))
        res_df[TTTR_OVN] = list(merge_df[tttr_ovn_union_col_name].itertuples(index=False, name=None))
        res_df[TTTR_WE] = list(merge_df[tttr_we_union_col_name].itertuples(index=False, name=None))
        return res_df

#function for reformating merged travel_time list from each month to get ready for the next step
def format_cell(x):
    travel_time_lst = []
    for i in x:
        i = i.replace('[', r'').replace(']', r'') #replace '[' and ']' charactors
        i = list(map(float, i.split(','))) #convert string to float, split by ','
        travel_time_lst.append(i)

    return list(chain.from_iterable(travel_time_lst)) #flat 3D to 2D list

#function for LOTTR calculation
def lottr_tttr_cal(x, percen_numerator, percen_denominator):
    num = np.nanpercentile(x, percen_numerator, interpolation = interpolation) 
    denom = np.nanpercentile(x, percen_denominator, interpolation = interpolation)
    return num/denom

#define a function to create the 'Reliability' column
def reliability_col(row):
    if (row['LOTTR_AMP'] >= 1.5) or (row['LOTTR_MIDD'] >= 1.5) or (row['LOTTR_PMP'] >= 1.5) or (row['LOTTR_WE'] >= 1.5):
        return 'U'
    else:
        return 'R'

'''End of functions'''


# dash = [i for i, v in enumerate(month_input) if v == '-']


'''variable names declaration'''
#declare constant number variables
year = 2017

directional_factor_number = 0.5 
occupancy_factor_number = 1.7


#declare the percentile values used in LOTTR and TTTR
lottr_percen_numerator = 80
lottr_percen_denominator = 50
tttr_percen_numerator = 95
tttr_percen_denominator = 50
interpolation = 'nearest'

MAX_TTTR = 'MAX_TTTR'
reliability = 'reliability'
tmc = 'tmc'
miles = 'miles'
faciltype = 'faciltype'
aadt = 'aadt'
occupancy_factor = 'occupancy_factor'
days_month = 'days_month'
days_year = 'days_year'
directional_factor = 'directional_factor'



#get number of days in a year by checking number of days in Feb
#check number of days in Feb
Feb = 2
days_Feb = calendar.monthrange(year,Feb)[1]
#get number of days in the given year
if days_Feb == 28:
    days_year_number = 365
else:
    days_year_number = 366

#create a lookup dic for 12 months
month_num = list(range(1,13))
month_abbr = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', \
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_num_abbr_lookup = dict(zip(month_num, month_abbr))

#define folder names that used for storing the csv files for annual analysis 
#same as the ones defined in the script for monthly data
#define folder names for storing csv files used for annual analysis
foldername_lottr_inter_csv = 'Annual_Analysis/' + 'lottr_inter_' + str(year) 
foldername_lottr_noninter_csv = 'Annual_Analysis/' + 'lottr_noninter_' + str(year) 
foldername_tttr_inter_csv = 'Annual_Analysis/' + 'tttr_inter_' + str(year)

#define directory
mypath = '/Users/wolfie/Desktop/SPC/NPMRDS/' #the main path
args_lottr_inter = (mypath, foldername_lottr_inter_csv)
args_lottr_noninter = (mypath, foldername_lottr_noninter_csv)
args_tttr_inter = (mypath, foldername_tttr_inter_csv)

#delcare column names for both lottr and tttr
tmc, LOTTR_AMP, LOTTR_MIDD, LOTTR_PMP, LOTTR_WE = 'tmc', 'LOTTR_AMP','LOTTR_MIDD', 'LOTTR_PMP','LOTTR_WE'
tmc, TTTR_AMP, TTTR_MIDD, TTTR_PMP, TTTR_OVN, TTTR_WE = 'tmc', 'TTTR_AMP', 'TTTR_MIDD', 'TTTR_PMP', 'TTTR_OVN', 'TTTR_WE'
#define column names for proessing multi-month/annual data
column_names_lottr = [LOTTR_AMP, LOTTR_MIDD, LOTTR_PMP, LOTTR_WE]
column_names_tttr = [TTTR_AMP, TTTR_MIDD, TTTR_PMP, TTTR_OVN, TTTR_WE]
#define column names for extracting final result
column_names_lottr_final = [tmc, LOTTR_AMP, LOTTR_MIDD, LOTTR_PMP, LOTTR_WE, reliability]
column_names_tttr_final = [tmc, TTTR_AMP, TTTR_MIDD, TTTR_PMP, TTTR_OVN, TTTR_WE, MAX_TTTR]

#ask user to input months they'd like to perform analysis on
whole_or_not = input('Would you like to perform the NPMRDS analysis for the whole year (y/n)?')

if whole_or_not == 'y':
    #get all monthly intermediate csv files name in a given directory
    lottr_inter_csv = []
    for (dirpath, dirnames, filenames) in walk(os.path.join(*args_lottr_inter)):
        lottr_inter_csv.extend(filenames)
        break
    lottr_noninter_csv = []
    for (dirpath, dirnames, filenames) in walk(os.path.join(*args_lottr_noninter)):
        lottr_noninter_csv.extend(filenames)
        break
    tttr_inter_csv = []
    for (dirpath, dirnames, filenames) in walk(os.path.join(*args_tttr_inter)):
        tttr_inter_csv.extend(filenames)
        break
    ##alternative approach with os.listdir()
    # from os import listdir
    # from os.path import isfile, join
    # lottr_inter_csv = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    #remove hidden files 
    lottr_inter_csv = [i for i in lottr_inter_csv if not i.startswith('.')]
    lottr_noninter_csv = [i for i in lottr_noninter_csv if not i.startswith('.')]
    tttr_inter_csv = [i for i in tttr_inter_csv if not i.startswith('.')]
    
    #number of days in selected months = number of days in the entire year
    days_month_number = days_year_number 
    
    month_selection = list(range(1,13)) #select 12-month
    
elif whole_or_not == 'n':
    month_input = input('Please enter the month number(s) you would like to run analysis on: ' + '\n' \
                       + 'e.g. "1 3-5" means Jan March-May' + '\n')
    #apply get_month_input() function to get the selected filenames
    lottr_inter_csv, lottr_noninter_csv, tttr_inter_csv, month_selection = get_month_input()
    
    #get number of days in selected months
    days_month_number =0
    for mon in month_selection: 
        days_month_number += calendar.monthrange(year,mon)[1]
    
    

#read all monthly intermediate csv files
##lottr interstate
read_csv_lottr_inter = {}
for index, name in enumerate(lottr_inter_csv): 
    read_csv_lottr_inter[name] = pd.read_csv(os.path.join(*(foldername_lottr_inter_csv, name)))

##lottr noninterstate
read_csv_lottr_noninter = {}
for index, name in enumerate(lottr_noninter_csv): 
    read_csv_lottr_noninter[name] = pd.read_csv(os.path.join(*(foldername_lottr_noninter_csv, name)))

##tttr interstate
read_csv_tttr_inter = {}
for index, name in enumerate(tttr_inter_csv): 
    read_csv_tttr_inter[name] = pd.read_csv(os.path.join(*(foldername_tttr_inter_csv, name)))                                          

#save all filenames in a list
filenames_lottr_inter = list(read_csv_lottr_inter.keys())
filenames_lottr_noninter = list(read_csv_lottr_noninter.keys())
filenames_tttr_inter = list(read_csv_tttr_inter.keys())
                                            
#get all dataframes in a list
lottr_inter_dfs = [] #lottr - inter
for i in list(range(0,len(filenames_lottr_inter))):
    lottr_inter_dfs.append(read_csv_lottr_inter[filenames_lottr_inter[i]])

lottr_noninter_dfs = [] #lottr - noninter
for i in list(range(0,len(filenames_lottr_noninter))):
    lottr_noninter_dfs.append(read_csv_lottr_noninter[filenames_lottr_noninter[i]])

tttr_inter_dfs = [] #tttr - inter
for i in list(range(0,len(filenames_tttr_inter))):
    tttr_inter_dfs.append(read_csv_tttr_inter[filenames_tttr_inter[i]])

#outer merge all the csv read files on tmc numbers
merge_lottr_inter = reduce(lambda left,right: pd.merge(left,right,how='outer', on='tmc'), lottr_inter_dfs)
merge_lottr_noninter = reduce(lambda left,right: pd.merge(left,right,how='outer', on='tmc'), lottr_noninter_dfs)
merge_tttr_inter = reduce(lambda left,right: pd.merge(left,right,how='outer', on='tmc'), tttr_inter_dfs)

#group same column names together - merge multiple monthly lists into one
find_same_col_name_lottr_inter = {} #lottr_inter
for col_name in column_names_lottr[1:]:
    find_same_col_name_lottr_inter[col_name] = [col for col in merge_lottr_inter if col.startswith(col_name)]

find_same_col_name_lottr_noninter = {} #lottr_noninter
for col_name in column_names_lottr[1:]:
    find_same_col_name_lottr_noninter[col_name] = [col for col in merge_lottr_noninter if col.startswith(col_name)]
    
find_same_col_name_tttr_inter = {} #tttr
for col_name in column_names_tttr[1:]:
    find_same_col_name_tttr_inter[col_name] = [col for col in merge_tttr_inter if col.startswith(col_name)]

#Generate dataframe with tmc column for final output
#constract dictionary by getting tmc values from merged dataframe (selected months / entire year)
tmc_lottr_inter = {tmc: merge_lottr_inter.tmc.values.tolist(), }
tmc_lottr_noninter = {tmc: merge_lottr_noninter.tmc.values.tolist()}
tmc_tttr_inter = {tmc: merge_tttr_inter.tmc.values.tolist()}

#constract dataframe from dictionary
res_lottr_inter = pd.DataFrame(data = tmc_lottr_inter)
res_lottr_noninter = pd.DataFrame(data = tmc_lottr_noninter)
res_tttr_inter = pd.DataFrame(data = tmc_tttr_inter)

#group col names by each time segment for selected months
#lottr -  inter & noninter
lottr_amp_union_col_name = [col for col in merge_lottr_inter if col.startswith(LOTTR_AMP)] 
lottr_midd_union_col_name = [col for col in merge_lottr_inter if col.startswith(LOTTR_MIDD)]
lottr_pmp_union_col_name = [col for col in merge_lottr_inter if col.startswith(LOTTR_PMP)]
lottr_we_union_col_name = [col for col in merge_lottr_inter if col.startswith(LOTTR_WE)]

#tttr
tttr_amp_union_col_name = [col for col in merge_tttr_inter if col.startswith(TTTR_AMP)]
tttr_midd_union_col_name = [col for col in merge_tttr_inter if col.startswith(TTTR_MIDD)]
tttr_pmp_union_col_name = [col for col in merge_tttr_inter if col.startswith(TTTR_PMP)]
tttr_ovn_union_col_name = [col for col in merge_tttr_inter if col.startswith(TTTR_OVN)]
tttr_we_union_col_name = [col for col in merge_tttr_inter if col.startswith(TTTR_WE)]
    
#apply merge_column(res_df, merge_df, lottr_or_tttr) function to lottr inter & noninter, and tttr dataframes
res_lottr_inter = merge_column(res_lottr_inter, merge_lottr_inter, 'lottr') #lottr-inter
res_lottr_noninter = merge_column(res_lottr_noninter, merge_lottr_noninter, 'lottr') #lottr-noninter
res_tttr_inter = merge_column(res_tttr_inter, merge_tttr_inter, 'tttr') #tttr - inter

#apply format_cell() function to each element x in dataframe except 'tmc' (1st col)
res_lottr_inter_sub = res_lottr_inter.iloc[:, 1:].applymap(lambda x: format_cell(x)) #lottr-inter
res_lottr_noninter_sub = res_lottr_noninter.iloc[:, 1:].applymap(lambda x: format_cell(x)) #lottr-noninter
res_tttr_inter_sub = res_tttr_inter.iloc[:, 1:].applymap(lambda x: format_cell(x)) #tttr-inter

#concat formated dataframe to tmc column
res_lottr_inter = pd.concat([res_lottr_inter.iloc[:, 0], res_lottr_inter_sub], axis=1) #lottr-inter
res_lottr_noninter = pd.concat([res_lottr_noninter.iloc[:, 0], res_lottr_noninter_sub], axis=1) #lottr-noninter
res_tttr_inter = pd.concat([res_tttr_inter.iloc[:, 0], res_tttr_inter_sub], axis=1) #tttr - inter

#applay LOTTR_cal function to each time segment col
#lottr-inter
for i in [LOTTR_AMP, LOTTR_MIDD, LOTTR_PMP, LOTTR_WE]:
    res_lottr_inter[i] = res_lottr_inter[i].to_frame().applymap(lambda travel_time: lottr_tttr_cal(travel_time, lottr_percen_numerator, lottr_percen_denominator))

#lottr-noninter
for i in [LOTTR_AMP, LOTTR_MIDD, LOTTR_PMP, LOTTR_WE]:
    res_lottr_noninter[i] = res_lottr_noninter[i].to_frame().applymap(lambda travel_time: lottr_tttr_cal(travel_time, lottr_percen_numerator, lottr_percen_denominator))
    
#tttr-inter
for i in [TTTR_AMP, TTTR_MIDD, TTTR_PMP, TTTR_OVN, TTTR_WE]:
    res_tttr_inter[i] = res_tttr_inter[i].to_frame().applymap(lambda travel_time: lottr_tttr_cal(travel_time, tttr_percen_numerator, tttr_percen_denominator))

#classify reporting segments by adding new column 'Reliability' based on the values of AMP, MIDD, PMP, WE
#Round lottr and tttr results dataframe to 2 decimal places
res_lottr_inter = res_lottr_inter.round(2) #lottr - inter
res_lottr_noninter = res_lottr_noninter.round(2) #lottr - noninter
res_tttr_inter = res_tttr_inter.round(2) #tttr - inter

#apply reliability_col() function to add 'Reliability' column for both interstate and non-interstate dataframes
res_lottr_inter[reliability] = res_lottr_inter.apply(lambda row: reliability_col(row), axis=1) # lottr - inter
res_lottr_noninter[reliability] = res_lottr_noninter.apply(lambda row: reliability_col(row), axis=1) # lottr - inter

#add 'MAX_TTTR' column for max TTTR value for each tmc
res_tttr_inter[MAX_TTTR] = res_tttr_inter[[TTTR_AMP, TTTR_MIDD, TTTR_OVN, TTTR_PMP, TTTR_WE]].max(axis=1)


Would you like to perform the NPMRDS analysis for the whole year (y/n)? y


TypeError: reduce() of empty sequence with no initial value

In [329]:
#read identification files for 12 months
identi_tp = {}
identi_t = {}
for mon in range(1,13):
    #define month format in file & folder names - two digits
    two_digit_format_month = "{:02d}".format(mon)

    #declare folder and file names
    folder_upper = '15_min_data' # Upper Data Folder
    folder_lower_tp = str(year)+'-'+str(two_digit_format_month)+'-SPC-Region-TMCs_Truck&Passenger' #Lower Data Folder - Truck&Passenger 
    folder_lower_t = str(year)+'-'+str(two_digit_format_month)+'-SPC-Region-TMCs_Truck' #Lower Data Folder - Truck 

    file_identification_tp = 'TMC_Identification.csv' #identification dataset - Truck&Passenger 
    file_identification_t = 'TMC_Identification.csv' #identification dataset - Truck

    #combine name segments
    #truck&passenger
    args_identification_tp = (folder_upper, folder_lower_tp, file_identification_tp)
    #truck
    args_identification_t = (folder_upper, folder_lower_t, file_identification_t)
    #save identification file names 
    identi_tp[mon] = args_identification_tp
    identi_t[mon] = args_identification_t

#read identification files for 12 months
iden_df_tp = {}
iden_df_t = {}
#read truck&passenger
for mon, file_path in identi_tp.items():
    iden_df_tp[mon] = pd.read_csv(os.path.join(*file_path)) 
#read truck
for mon, file_path in identi_t.items():    
    iden_df_t[mon] = pd.read_csv(os.path.join(*file_path))

for iden_file in iden_df_tp.values():
    iden_file[tmc]

#check tmc codes difference for 12-month
i = 2
count = 1
tmc_diff_dic = {}
while i < 13:
    #compare tmc code from Feb-Dec with Jan 
    tmc_difference = set(list(iden_df_t[1][tmc]))-set(list(iden_df_t[i][tmc])) 
    #there is tmc difference
    if len(tmc_difference) != 0:
#         #TO DO
        pass
    #there is no tmc difference 
    elif len(tmc_difference) == 0:
        i+=1
        count += 1
        continue

if count == 12: #no difference in tmc for 12-month, use identification file from Jan
    identification_tp = iden_df_tp[1] #truck&passenger
    identification_t = iden_df_t[1] #truck


'''Travel Time Reliability Measurement'''
#select useful columns for TTR calculation from identification df
iden_tp_seg = identification_tp[[tmc, miles, faciltype, aadt]]
iden_t_seg = identification_t[[tmc, miles, faciltype, aadt]]

#join identification table to get columns for TTR calculation
res_lottr_inter = pd.merge(res_lottr_inter, iden_tp_seg, on='tmc', how='left')
res_lottr_noninter = pd.merge(res_lottr_noninter, iden_t_seg, on='tmc', how='left')

#round 'miles' column to 3 decimal places
res_lottr_inter[miles] = res_lottr_inter[miles].round(3)
res_lottr_noninter[miles] = res_lottr_noninter[miles].round(3)

#add column 'ocuupancy_factor' 'days_month', and 'days_year' (self-defined column names)
res_lottr_inter[occupancy_factor] = occupancy_factor_number
res_lottr_inter[days_month] = days_month_number
res_lottr_inter[days_year] = days_year_number

res_lottr_noninter[occupancy_factor] = occupancy_factor_number
res_lottr_noninter[days_month] = days_month_number
res_lottr_noninter[days_year] = days_year_number

#add column 'directional_factor'
def add_col(row):
    if row[faciltype]==2 or row[faciltype]==6:
        return 0.5
    elif row[faciltype]==1:
        return 1

#apply add_col() function to add 'directional_factor' column
res_lottr_inter[directional_factor] = res_lottr_inter.apply(lambda row: add_col(row), axis=1)
res_lottr_noninter[directional_factor] = res_lottr_noninter.apply(lambda row: add_col(row), axis=1)

#calculate volumn
def TTR_cal_s1(row):
    v1 = row[aadt]*row[days_month]*row[directional_factor] #monthly volumn
    ttr_s1 = row[miles]*v1*row[occupancy_factor]
    return ttr_s1

def TTR_cal_s2(df): 
    #select 'R' and 'U' 
    r = df[df[reliability]=='R']
    u = df[df[reliability]=='U']
    #calculate r/(r+u)
    ttr_s1 = r['ttr_s1'].sum()/(r['ttr_s1'].sum() + u['ttr_s1'].sum())
    #turn result into percentage with 1 decimal place
    ttr = "{:.1%}".format(ttr_s1) 
    return ttr

#apply TTR_cal_s1 function
res_lottr_inter['ttr_s1'] = res_lottr_inter.apply(lambda row: TTR_cal_s1(row), axis=1)
res_lottr_noninter['ttr_s1'] = res_lottr_noninter.apply(lambda row: TTR_cal_s1(row), axis=1)

#get TTR result by applying TTR_cal_s2 function
TTR_inter = TTR_cal_s2(res_lottr_inter)
TTR_noninter = TTR_cal_s2(res_lottr_noninter)

print(TTR_inter, TTR_noninter)


92.2% 85.5%


In [335]:
#select columns for final outputs
res_lottr_inter_final = res_lottr_inter[column_names_lottr_final]
res_lottr_noninter_final = res_lottr_noninter[column_names_lottr_final]
res_tttr_inter_final = res_tttr_inter[column_names_tttr_final]

# Get the number of rows for each dataframe in order to place the totals
number_rows1 = len(res_lottr_inter.index) 
number_rows2 = len(res_lottr_noninter.index)
number_rows3 = len(res_tttr_inter.index)

#define final report name base on user month input
if len(month_selection) != 12: #user doesn't select the entire year for analysis
    report_name = 'multi-month_report_'+ month_input+'.xlsx'
#     month_for_output = month_num_abbr_lookup
elif len(month_selection) == 12: #user select entire year for analysis
    report_name = 'annual_report_'+ str(year)+'.xlsx'
    month_for_output = str(year)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
# write dataframes to excel sheets
res_lottr_inter_final.to_excel(writer, index=False, sheet_name='LOTTR_inter')
res_lottr_noninter_final.to_excel(writer, index=False, sheet_name='LOTTR_noninter')
res_tttr_inter_final.to_excel(writer, index=False, sheet_name='TTTR_inter')
res_lottr_inter.to_excel(writer, index=False, sheet_name='LOTTR_inter_intermediate')
res_lottr_noninter.to_excel(writer, index=False, sheet_name='LOTTR_noninter_intermediate')
res_tttr_inter.to_excel(writer, index=False, sheet_name='TTTR_inter_intermediate')


# Get access to the workbook and sheet
workbook = writer.book
worksheet1 = writer.sheets['LOTTR_inter']
worksheet2 = writer.sheets['LOTTR_noninter']
worksheet3 = writer.sheets['TTTR_inter']
worksheet4 = writer.sheets['LOTTR_inter_intermediate']
worksheet5 = writer.sheets['LOTTR_noninter_intermediate']
worksheet6 = writer.sheets['TTTR_inter_intermediate']


# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

total_fmt = workbook.add_format({'align': 'right', 'num_format': '$#,##0',
                                 'bold': True, 'bottom':6})

# Format the columns by width and include number formats

# Account info columns
worksheet1.set_column('A:F', 16)
worksheet2.set_column('A:F', 16)
worksheet3.set_column('A:G', 16)
worksheet4.set_column('A:Q', 16)
worksheet5.set_column('A:Q', 16)
worksheet6.set_column('A:L', 16)


# Define our range for the color formatting
color_range1 = "F2:F{}".format(number_rows1+1)
color_range2 = "F2:F{}".format(number_rows2+1)


worksheet1.conditional_format(color_range1, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"U"',
                                        'format':   format1})

worksheet1.conditional_format(color_range1, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"R"',
                                        'format':   format2})

worksheet2.conditional_format(color_range2, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"U"',
                                        'format':   format1})

worksheet2.conditional_format(color_range2, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"R"',
                                        'format':   format2})

worksheet4 = workbook.add_worksheet('Data_info')
worksheet5 = workbook.add_worksheet('Column_name_info')

#text box for general data information
text1 =  str(year) \
    + '\nTTR interstate = ' + str(TTR_inter) \
    + '\nTTR noninterstate = ' + str(TTR_noninter) \
    + '\nTotal tmcs: '+ str(len(identification_tp.tmc)) 
    
#text box for final output column names lookup
text2 = 'LOTTR_AMP: weekdays (Monday - Friday) 6:00 AM - 10:00 AM' \
    + '\nLOTTR_MIDD: weekdays (Monday - Friday) 10:00 AM - 4:00 PM' \
    + '\nLOTTR_PMP: weekdays (Monday - Friday) 4:00 PM - 8:00 PM' \
    + '\nLOTTR_WE: weekends (Saturday - Sunday) 6:00 AM - 8:00 PM' \
    + '\n\nTTTR_AMP: weekdays (Monday - Friday) 6:00 AM - 10:00 AM' \
    + '\nTTTR_MIDD: weekdays (Monday - Friday) 10:00 AM - 4:00 PM' \
    + '\nTTTR_PMP: weekdays (Monday - Friday) 4:00 PM - 8:00 PM' \
    + '\nTTTR_OVN: everyday (Sunday - Saturday) 8:00 PM - 6:00 AM' \
    + '\nTTTR_WE: weekends (Saturday - Sunday) 6:00 AM - 8:00 PM' 


#define format
options = {
    'width': 800,
    'height': 400,
    'x_offset': 10,
    'y_offset': 10,

    'font': {'color': 'black',
             'size': 14},
    'align': {'vertical': 'middle',
              'horizontal': 'center'
              },
    'line': {'color': '#C6EFCE'},
}

#insert textbox into sheets
worksheet4.insert_textbox('B2', text1, options)
worksheet5.insert_textbox('B2', text2, options)

writer.save()
print('completed' + str(month_selection))


completed[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
